In [226]:
import joblib
import copy
from collections import Counter
import pandas as pd
import numpy as np
import os
import pickle
from xgboost import XGBRegressor
from catboost import Pool, CatBoostRegressor
from smt.surrogate_models import RBF,KRG,LS
from sklearn.feature_selection import f_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern
from sklearn.gaussian_process import kernels
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold,GridSearchCV
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,NuSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import plotly.colors as colors
import plotly.express as px
from plotly.subplots import make_subplots
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.util.normalization import denormalize
from pymoo.visualization.scatter import Scatter
from smt.surrogate_models import RBF,KRG,LS
from scipy.interpolate import RBFInterpolator
# from smt.surrogate_models.rbf import RBF
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.problems import get_problem
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.isres import ISRES
import warnings
import joblib
import seaborn as sns
warnings.filterwarnings("ignore")
np.set_printoptions(suppress = True)

In [130]:
# pd.read_csv(r'./data/data_1110.csv', na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})

In [227]:
def getData():
    '''读取数据'''
    data1 = pd.read_csv('./data/data_1110.csv', na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    data2 = pd.read_csv('./data/data_1205.csv',na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    data = pd.concat([data1,data2], axis=0).iloc[:,1:]
    data['flag'] = ['F3' for i in range(data.shape[0])]
    pdata = pd.read_csv('./data/data_0104.csv', na_values={'dist_out_in':-1,'dist_out_mid':-1,'dist_mid_in':-1,'dist_in_hard':-1}).iloc[:,1:]
    pdata['flag'] = ['F2' for i in range(pdata.shape[0])]
    rawData = pd.concat([data, pdata], axis=0).iloc[:,1:].reset_index(drop=True)
    return rawData.iloc[:,:-2], rawData['HIC15'], rawData['flag']

In [228]:
rawData, rawLabel, rawFlag = getData()

In [229]:
rawData.shape

(537, 57)

In [230]:
rawFlag.value_counts()

flag
F3    356
F2    181
Name: count, dtype: int64

# 数据预处理

## 删除缺失率大于80%的数据列

In [231]:
(rawData.isnull().sum()/rawData.shape[0]).sort_values(ascending=False)[:5]

dist_mid_in     0.830540
dist_out_mid    0.826816
dist_in_hard    0.329609
dist_out_in     0.214153
Y_S_out         0.000000
dtype: float64

In [232]:
rawData.drop(columns=['dist_mid_in','dist_out_mid'], inplace=True)

In [233]:
rawData.shape

(537, 55)

## 均值填充缺失处理和方差筛选

In [234]:
def imputerVar(data, threshold=1):
    '''缺失处理和方差筛选'''
    simpleImputer = SimpleImputer().fit(data)
    joblib.dump(simpleImputer, './models/simpleImputer.pkl')
    siData = simpleImputer.transform(data)
    siData = pd.DataFrame(siData,columns=data.columns)
    varModel = VarianceThreshold(threshold=threshold).fit(siData)
#     print(data.shape)
    print(set(varModel.feature_names_in_)-set(varModel.get_feature_names_out()))
    siData = varModel.transform(siData)
    siData = pd.DataFrame(siData, columns=varModel.get_feature_names_out())
    inFeatures = siData.columns
    return siData, inFeatures

In [235]:
rawData, inFeatures = imputerVar(rawData, 0)

{'T_mid_R', 'JR_X', 'delta_in', 'T_S_out', 'T_S_mid_R', 'JR_Z', 'delta_out', 'Y_S_in', 'Y_S_out', 'JR_Y', 'delta_mid_F', 'T_S_in', 'delta_mid_R', 'T_S_mid_F', 'head_V', 'T_mid_F', 'Y_S_mid_R', 'T_in', 'T_out', 'Y_S_mid_F'}


In [236]:
rawData.shape

(537, 35)

## 异常值检测

In [237]:
def solveoutliers(isExec, data, label, flag, contamination):
    '''是否处理异常值'''
    def isoForest(data, label, flag, contamination=0.1):
        '''孤立森林去除异常值'''
        isoData = copy.deepcopy(data)
        clf = IsolationForest(max_samples='auto', random_state=0, max_features=1,contamination=contamination).fit(isoData)
        myIsoDataIndex = clf.predict(isoData)
        print(Counter(myIsoDataIndex))
#         print(myIsoDataIndex)
        data_ = isoData[myIsoDataIndex==1]
        label_ = label[myIsoDataIndex==1]
        flag_ = flag[myIsoDataIndex==1]
        isoData = isoData[myIsoDataIndex==-1]
        isoLabel = label[myIsoDataIndex==-1]
        isoFlag = flag[myIsoDataIndex==-1]
        pd.concat([isoData,isoLabel,isoFlag], axis=1).to_csv('./data/significantSample.csv')
        return data_, label_, flag_
    if isExec:
        return isoForest(data,label, flag, contamination)
    return data, label, flag

In [238]:
data, label, flag = solveoutliers(False, rawData, rawLabel, rawFlag, 0.05)

In [239]:
data.shape

(537, 35)

# 可视化分析训练数据测试数据分布

In [240]:
def showVisual(data, label, flag ):
    '''训练集和测试集数据可视化'''
    columns = ['point_Z',
             'point_Y',
             'bonnet_YR',
#              'Izz',
#              'Ixx',
             'Iyy'
             ]
    visData = copy.deepcopy(data[columns])
    visData['label'] = label
    visData['flag'] = flag
    pp = sns.pairplot(visData, hue='flag',markers=["o", "s"], corner=True, diag_kind="auto",kind='reg')
#     sns.pairplot(visData,hue='flag')
    pp.fig.suptitle('The linear relationship between variables under different battery types')
    # g.map_lower(sns.kdeplot, levels=4, color=".2")
    plt.title('训练集和测试集之间关系图')
    plt.show()

In [241]:
# showVisual(data, label, flag)

In [242]:
data.groupby(flag)[['bonnet_YR','point_Z','bonnet_YL','Iyy']].mean()


,bonnet_YR,point_Z,bonnet_YL,Iyy
flag,,,,
F2,824.163473,825.491111,106.087513,20.766766
F3,799.769620,111.715113,800.556842,22.011294


# 数据标准化

In [243]:
def stdModel(data):
    '''数据标准化，然后拆分数据集'''
    stdModel = preprocessing.StandardScaler().fit(data)
    stdData = stdModel.transform(data)
    joblib.dump(stdModel,'./models/stdModel.pkl')
    data = pd.DataFrame(stdData, columns=data.columns)
    return data
# data = stdModel(data)

In [244]:
data.shape

(537, 35)

In [245]:
colNames = data.columns
colNames

Index(['point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_X', 'JL_Y', 'JL_Z', 'LL_X', 'LL_Y',
       'LL_Z', 'LR_X', 'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X',
       'B_LF_Y', 'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y',
       'B_RF_Z'],
      dtype='object')

# 数据拆分

In [246]:
def splitData(flag, rawLabel):
    '''数据拆分'''
    data['label'] = rawLabel
#     data['flag'] = flag
    trainData = data[flag=='F3']
    testData = data[flag=='F2']
#     x_train,x_test,y_train,y_test = train_test_split(data,label,test_size=0.1,random_state=420)
    yTrain = trainData.pop('label')
    xTrain = trainData
    yTest = testData.pop('label')
    xTest = testData
    return xTrain,xTest,yTrain,yTest 

In [247]:
xTrain,xTest,yTrain,yTest = splitData(rawFlag, rawLabel)

In [248]:
xTrain.shape

(356, 35)

# 创建模型

## 线性回归模型

In [251]:
def createLrModelSklearn():
    model = LinearRegression()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    print(model.coef_)
    joblib.dump(model, './models/lrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [252]:
lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()

modelname:LinearRegression(),
trainR2:0.793,
testR2:-1.3790121496140099e+18
[-4.06484817e-01 -1.79974736e+01 -4.38218369e-02  2.77261348e-02
 -8.19182961e-02 -1.08224680e-01 -1.14263510e+01 -3.95862593e-01
 -1.00507005e+00  2.87280827e+02  2.65205277e+02 -2.81831590e+02
  3.16758566e+01  7.31381103e+01 -2.59976192e+10 -1.23300674e+09
  1.62047748e+07  5.70575870e+10  1.34147335e+11  8.94399631e+09
  5.70575870e+10 -1.65042559e+10  8.94399631e+09 -8.45310646e+09
 -1.02204926e+11  9.66647241e+09  3.91109040e+10 -6.84165884e+10
  1.84872031e+10 -8.86935442e+10  3.90636252e+10  6.09080198e+09
 -4.03358156e+10 -5.27672463e+10  1.51265142e+10]


# 支持向量机

In [253]:
def createSVRModelSklearn():
    '''创建支持向量机回归模型'''
    model = SVR()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/svrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [254]:
svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()

modelname:SVR(),
trainR2:-0.104,
testR2:-0.194


## 特征重要性函数编写

In [255]:
def getFeaturesImportance(model, n, flag, colNames):
    '''获取特征重要性排序,model:模型名字，n：过滤阈值，flag:模型名字，colNames，列名字'''
    randomForestFeaturesImportance = dict(zip(colNames, model.feature_importances_))
    sorted(randomForestFeaturesImportance.items(), key=lambda d: d[1], reverse=True)
    randomForestFeaturesImportance = pd.DataFrame.from_dict(randomForestFeaturesImportance,orient='index') \
                                                            .rename(columns={'index':'featureName',0:'score'}) \
                                                            .sort_values(by='score', ascending=False)
    randomForestFeaturesImportance = randomForestFeaturesImportance[randomForestFeaturesImportance['score']>n].reset_index()
    randomForestFeaturesImportance.to_csv('./data/'+flag+'FeaturesImportance.csv', index=False)
    return randomForestFeaturesImportance

## 随机森林

In [256]:
def createRandomForestModelSklearn():
    '''创建随机森林模型'''
    model = RandomForestRegressor(random_state=1)
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/randomForestModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [258]:
randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()


modelname:RandomForestRegressor(random_state=1),
trainR2:0.984,
testR2:0.524


In [259]:
randomForestFeaturesImportance = getFeaturesImportance(randomForestModelSklearn,0.,'randomForest',colNames)


## XGB模型

In [260]:
def createXgbModelSklearn():
    '''创建XGB模型'''
    model = XGBRegressor()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/xgbModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [261]:
xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()

modelname:XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...),
trainR2:1.0,
testR2:0.546


In [262]:
randomForestFeaturesImportance = getFeaturesImportance(xgbModelSklearn,0.,'xgbModel',colNames)


## CatBoostRegressor

In [263]:
def createCatBoostModel():
    '''创建CatBoost模型'''
    # initialize Pool
    trainPool = Pool(xTrain,
                      yTrain)
    testPool = Pool(xTest) 

    # specify the training parameters 
    model = CatBoostRegressor(silent=True)
    #train the model
    model.fit(trainPool)
    # make the prediction using the resulting model
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/catBoostModel.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [264]:
catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()

modelname:<catboost.core.CatBoostRegressor object at 0x0000028FBA544E90>,
trainR2:0.999,
testR2:0.484


In [265]:
catBoostFeaturesImportance = getFeaturesImportance(catBoostModelSklearn,0.,'catBoost',colNames)


# 加载所有模型并预测

In [266]:
predData = pd.read_csv('./data/preData.csv')[colNames].values
predData.shape

(3, 35)

In [269]:
def loadModelAndPred(data,yTrue=None):
    '''加载预测模型进行验证'''
    lryPredSklearn = joblib.load('./models/lrModelSklearn.pkl').predict(data)
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl').predict(data)
    svryPredSklearn = joblib.load('./models/svrModelSklearn.pkl').predict(data)
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl').predict(data)
    catBoostyPred = joblib.load('./models/catBoostModel.pkl').predict(data)
    # 收集预测结果形成csv文件
    allModelPreds = {
                    '线性回归':lryPredSklearn,
                    '随机森林':randomForestyPredSklearn,
                    '支持向量机':svryPredSklearn,
                    'xgboost':xgbyPredSklearn,
                    'catBoost':catBoostyPred
    }
    allModelPreds = pd.DataFrame(allModelPreds).round(3)
    if yTrue is not None:
        allModelPreds['真实值'] = yTrue
        allModelPreds.to_csv('./data/allModelPredsF2F3.csv', encoding='gbk')
    return allModelPreds
# 
# allModelPreds = loadModelAndPred(predData)
allModelPreds = loadModelAndPred(xTest.values,yTrue=yTest.values)
allModelPreds.head()

,线性回归,随机森林,支持向量机,xgboost,catBoost,真实值
0,-6.658024e+11,779.699,728.841,551.940002,756.755,636.0
1,-6.658024e+11,834.579,729.802,626.465027,800.761,644.0
2,-6.658024e+11,830.809,729.547,678.497009,893.316,686.0
3,-6.658024e+11,789.890,729.223,544.500000,737.045,702.0
4,-6.658024e+11,775.302,728.788,574.405029,875.731,746.0


# 求百分比1-abs(yTrue-yTest)/yTrue

In [270]:
allModelPredsPercentage = pd.DataFrame()
allModelPredsPercentage['xgboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['xgboost']).abs()/allModelPreds['真实值']
allModelPredsPercentage['catboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['catBoost']).abs()/allModelPreds['真实值']
allModelPredsPercentage.to_csv('./data/allModelPredsPercentF2F3.csv', encoding='gbk')

## 求所有的R2数据

In [272]:

def createAllModel():
    '''集中统一调用,
    线性回归模型	支持向量机模型	随机森林模型	XGB模型-->catBoost模型
    '''
    print('*'*90)
    lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()
    print('*'*90)
    svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()
    print('*'*90)
    randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()
    print('*'*90)
    xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()
    print('*'*90)
    catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()    
    allModelR2 = {
                    '线性回归':[lrR2TrainSklearn, lrR2TestSklearn],
                    '支持向量机':[svrR2TrainSklearn, svrR2TestSklearn],
                    '随机森林':[randomForestR2TrainSklearn, randomForestR2TestSklearn],
                    'xgboost':[xgbR2TrainSklearn, xgbR2TestSklearn],
                    'catBoost':[catBoostR2TrainSklearn, catBoostR2TestSklearn]
    }
    allModelR2 = pd.DataFrame(allModelR2,index=['训练集', '测试集'])
    allModelR2.to_csv('./data/allModelR2F2F3.csv', encoding='gbk')
    allModelMae = {
                    '线性回归':[lrMaeTrainSklearn, lrMaeTestSklearn],
                    '支持向量机':[svrMaeTrainSklearn, svrMaeTestSklearn],
                    '随机森林':[randomForestMaeTrainSklearn, randomForestMaeTestSklearn],
                    'xgboost':[xgbMaeTrainSklearn, xgbMaeTestSklearn],
                    'catBoost':[catBoostMaeTrainSklearn, catBoostMaeTestSklearn]
    }
    allModelMae = pd.DataFrame(allModelMae,index=['训练集', '测试集'])
    allModelMae.to_csv('./data/allModelMaeF2F3.csv', encoding='gbk')
    
    return allModelR2, allModelMae
createAllModel()    
  

******************************************************************************************
modelname:LinearRegression(),
trainR2:0.793,
testR2:-1.3790121496140099e+18
[-4.06484817e-01 -1.79974736e+01 -4.38218369e-02  2.77261348e-02
 -8.19182961e-02 -1.08224680e-01 -1.14263510e+01 -3.95862593e-01
 -1.00507005e+00  2.87280827e+02  2.65205277e+02 -2.81831590e+02
  3.16758566e+01  7.31381103e+01 -2.59976192e+10 -1.23300674e+09
  1.62047748e+07  5.70575870e+10  1.34147335e+11  8.94399631e+09
  5.70575870e+10 -1.65042559e+10  8.94399631e+09 -8.45310646e+09
 -1.02204926e+11  9.66647241e+09  3.91109040e+10 -6.84165884e+10
  1.84872031e+10 -8.86935442e+10  3.90636252e+10  6.09080198e+09
 -4.03358156e+10 -5.27672463e+10  1.51265142e+10]
******************************************************************************************
modelname:SVR(),
trainR2:-0.104,
testR2:-0.194
******************************************************************************************
modelname:RandomForestRegressor(ra

(             线性回归  支持向量机   随机森林  xgboost  catBoost
 训练集  7.930000e-01 -0.104  0.984    1.000     0.999
 测试集 -1.379012e+18 -0.194  0.524    0.546     0.484,
              线性回归    支持向量机     随机森林  xgboost  catBoost
 训练集  1.656050e+02  317.153   32.171    0.118     9.177
 测试集  6.658024e+11  388.497  244.333  212.074   259.815)

 ## 汇总模型所有特征重要性

In [ ]:
def getAllFeaturesImportance(colNames):
    '''加载预测模型进行验证'''
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl')
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl')
    catBoostyPred = joblib.load('./models/catBoostModel.pkl')
    # 汇总模型所有特征重要性
    randomForestFeaturesImportance = getFeaturesImportance(randomForestyPredSklearn, -1.,'randomForest',colNames)
    xgbFeaturesImportance = getFeaturesImportance(xgbyPredSklearn, -1.,'xgboost',colNames)
    catBoostFeaturesImportance = getFeaturesImportance(catBoostyPred, -1.,'catBoost',colNames)
    allFeaturesImportance = pd.concat([randomForestFeaturesImportance,xgbFeaturesImportance,catBoostFeaturesImportance], axis=1)
    return allFeaturesImportance

In [ ]:
getAllFeaturesImportance(colNames)

# Optuna模型调参

In [322]:
import optuna
import xgboost as xgb
import sklearn
import lightgbm as lgb
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [352]:
SEED = 42
np.random.seed(SEED)

In [301]:
def createStudy(Objective, studyName):
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=SEED),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
        storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
        study_name=studyName,
        load_if_exists=True
    )
    study.optimize(Objective, n_trials=10, timeout=600)
    return study

In [304]:
def lgbObjective(trial):
    dtrain = lgb.Dataset(xTrain, label=yTrain)
    dvalid = lgb.Dataset(xTest, label=yTest)

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_iterations":trial.suggest_int('num_iterations', 5, 20),
        "learning_rate":trial.suggest_float("learning_rate",0.01, 0.1, step=0.01)
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
    
    preds = gbm.predict(xTest)
#     pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.r2_score(yTest, preds)
    return accuracy

In [305]:
lgbStudy = createStudy(lgbObjective, 'lgbModel')

[I 2024-01-08 15:01:19,684] Using an existing study with name 'lgbModel' instead of creating a new one.
[I 2024-01-08 15:01:20,434] Trial 12 finished with value: -0.08197711719281431 and parameters: {'num_iterations': 20, 'learning_rate': 0.01}. Best is trial 1 with value: -0.2574107579036913.
[I 2024-01-08 15:01:21,193] Trial 13 finished with value: -0.2574107579036913 and parameters: {'num_iterations': 20, 'learning_rate': 0.09999999999999999}. Best is trial 1 with value: -0.2574107579036913.
[I 2024-01-08 15:01:21,946] Trial 14 finished with value: -0.2574107579036913 and parameters: {'num_iterations': 20, 'learning_rate': 0.09999999999999999}. Best is trial 1 with value: -0.2574107579036913.
[I 2024-01-08 15:01:22,634] Trial 15 finished with value: -0.24691550782434035 and parameters: {'num_iterations': 18, 'learning_rate': 0.09999999999999999}. Best is trial 1 with value: -0.2574107579036913.
[I 2024-01-08 15:01:23,324] Trial 16 finished with value: -0.19848651233388237 and parame

In [294]:
def xgbObjective(trial):
#     data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(xTrain, label=yTrain)
    dvalid = xgb.DMatrix(xTest, label=yTest)

    param = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    
    preds = bst.predict(dvalid)
#     pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.r2_score(yTest, preds)
    return accuracy

In [295]:
xgbStudy = createStudy(xgbObjective, 'xgbModel')

[I 2024-01-08 14:51:25,487] Using an existing study with name 'xgbModel' instead of creating a new one.


[0]	validation-rmse:567.96476
[1]	validation-rmse:581.46632
[2]	validation-rmse:579.29618
[3]	validation-rmse:585.16356
[4]	validation-rmse:597.88962
[5]	validation-rmse:602.37671
[6]	validation-rmse:602.28196
[7]	validation-rmse:606.36210
[8]	validation-rmse:607.67594
[9]	validation-rmse:610.76459


[I 2024-01-08 14:51:26,163] Trial 100 finished with value: -0.1743087464455162 and parameters: {'booster': 'gbtree', 'lambda': 0.016050876683682797, 'alpha': 0.2435565924206753}. Best is trial 72 with value: -0.39815368665345674.


[0]	validation-rmse:565.55448
[1]	validation-rmse:567.68451
[2]	validation-rmse:572.77833
[3]	validation-rmse:589.33160
[4]	validation-rmse:595.40503
[5]	validation-rmse:606.67786
[6]	validation-rmse:614.45560
[7]	validation-rmse:621.36512
[8]	validation-rmse:630.22419
[9]	validation-rmse:637.49833


[I 2024-01-08 14:51:26,779] Trial 101 finished with value: -0.27935978078979784 and parameters: {'booster': 'gbtree', 'lambda': 0.14879658889779085, 'alpha': 0.41953664007695696}. Best is trial 72 with value: -0.39815368665345674.


...

[I 2024-01-08 14:51:32,040] Trial 107 finished with value: -0.36552380226416026 and parameters: {'booster': 'gbtree', 'lambda': 0.11619881700677846, 'alpha': 3.4164085154724293e-06}. Best is trial 72 with value: -0.39815368665345674.


[0]	validation-rmse:565.67838
[1]	validation-rmse:575.89077
[2]	validation-rmse:585.16105
[3]	validation-rmse:598.60760
[4]	validation-rmse:607.73913
[5]	validation-rmse:609.91528
[6]	validation-rmse:613.57554
[7]	validation-rmse:615.16209
[8]	validation-rmse:618.30621
[9]	validation-rmse:624.26560


[I 2024-01-08 14:51:32,850] Trial 108 finished with value: -0.22679898191926862 and parameters: {'booster': 'gbtree', 'lambda': 0.12422289780170334, 'alpha': 2.582283340560526e-06}. Best is trial 72 with value: -0.39815368665345674.


[0]	validation-rmse:567.96520
[1]	validation-rmse:581.43543
[2]	validation-rmse:579.24539
[3]	validation-rmse:585.08876
[4]	validation-rmse:597.76856
[5]	validation-rmse:602.35421
[6]	validation-rmse:602.24938
[7]	validation-rmse:606.27561
[8]	validation-rmse:607.58206
[9]	validation-rmse:610.66513


[I 2024-01-08 14:51:33,624] Trial 109 finished with value: -0.17392632180384893 and parameters: {'booster': 'gbtree', 'lambda': 0.027242976417725107, 'alpha': 0.0021775336942768068}. Best is trial 72 with value: -0.39815368665345674.


In [ ]:
xgbStudy.best_params